In [29]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from xgboost import XGBRanker
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
sns.set()
import warnings
warnings.filterwarnings("ignore")
import torch.nn as nn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from adapt.instance_based import KLIEP


In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.cross_decomposition import CCA
from sklearn.cross_decomposition import PLSCanonical
from sklearn.cross_decomposition import PLSSVD
from operator import itemgetter
def softmax(x):
    return(np.exp(x)/np.exp(x).sum())

def my_custom_loss_func_exp(y_true, y_pred):
    return spearmanr(np.exp(y_true), np.exp(y_pred)).correlation  

def my_custom_loss_func(y_true, y_pred):
    #y_pred = np.exp(np.array(y_pred))
    return spearmanr(y_true, y_pred).correlation

def loss(y_true,y_pred):
    return np.corrcoef(y_true,y_pred)[0][1]

def select_columns(X, y, test_size = 0.33, size_features= 30, state = 100):
    dic = dict({})
    for i in range(100,100+state):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i)
        X_train = np.array(X_train)
        X_test = np.array(X_test)
        all_corr = []
        for i in range(X_train.shape[1]):
            cor = my_custom_loss_func(y_train,X_train[:,i])
            all_corr.append(cor)
            
        all_corr = np.array(all_corr)
        cols = np.argsort(all_corr)[::-1]
        cols_keep = cols[:size_features]
        for i in cols_keep:
            if(i in dic):
                dic[i]+=1
            else:
                dic[i] = 1
   
    s = sorted(dic.items(), key=itemgetter(1),reverse=1)
    print(s)
    cols_keep = []
    for i in range(size_features):
        cols_keep.append(s[i][0])
    return np.array(cols_keep)

def evaluate_model_mean(X, y, model, stratify ,  test_size=0.33, loss = my_custom_loss_func, state = 100, size_features = 30):
    
    score = []
    for i in range(state):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=i, stratify=stratify)
        
        '''
        mask = (X_test['COUNTRY']==-1)
        X_test_FR = X_test[mask].copy()
        y_train = np.array(y_train)
        y_test_FR = np.array(y_test)[np.array(mask)].copy()
        
        
        mask = (X_test['COUNTRY']==1)
        X_test_DE = X_test[mask].copy()
        y_test_DE = np.array(y_test)[np.array(mask)].copy()
        '''
    
        X_train = np.array(X_train)
        X_test = np.array(X_test)
        '''
        X_test_FR = np.array(X_test_FR)
        X_test_DE = np.array(X_test_DE)
        '''
        #model_pca = PCA(n_components=X_train.shape[1])
        #model_pca.fit(X_train)
        #X_train = model_pca.transform(X_train)
        #X_test = model_pca.transform(X_test)
                
        
        model.fit(X_train,y_train)
        train_score = loss(y_train, model.predict(X_train))
        y_predict = model.predict(X_test)
        test_score = loss(y_test , y_predict.reshape(-1))
        '''
        y_predict_FR = model.predict(X_test_FR)
        test_score_FR = loss(np.array(y_test_FR) , y_predict_FR.reshape(-1))
        
        y_predict_DE = model.predict(X_test_DE)
        test_score_DE = loss(np.array(y_test_DE) , y_predict_DE.reshape(-1))
        '''
        score.append([test_score])#,test_score_FR,test_score_DE])
        
    return np.mean(np.array(score),axis=0)


def evaluate_model_mean_lgb(X, y, model, stratify ,  test_size=0.33, loss = my_custom_loss_func, state = 100, size_features = 30):

    score = []
    for i in range(state):
        import lightgbm as lgbm

        X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X, y, test_size=0.3,random_state=i, stratify=stratify)
        X_train_m, X_val_m, y_train_m, y_val_m = train_test_split(X_train_m, y_train_m, test_size=0.2,random_state=i)

        lgbm_parameters ={
        'n_estimators': 10000,
        'learning_rate': 0.0941884136011224,
        'num_leaves': 180,
        'max_depth': 12,
        'min_data_in_leaf': 200,
        'lambda_l1': 0.01,
        'lambda_l2': 1,
        'min_gain_to_split': 8.756413243587552,
        'bagging_fraction': 0.6000000000000001,
        'bagging_freq': 1,
        'feature_fraction': 0.5}

        model = lgbm.LGBMRegressor(objective="regression", **lgbm_parameters)
        model.fit(
            X_train_m,
            y_train_m,
            eval_set=[(X_val_m,y_val_m)],
            eval_metric="l1",
            early_stopping_rounds=100
        )

        X_train_m = np.array(X_train_m)
        X_test_m = np.array(X_test_m)
        
        train_score = loss(y_train_m, model.predict(X_train_m))
        y_predict = model.predict(X_test_m)
        test_score = loss(y_test_m , y_predict.reshape(-1))
        
        score.append([test_score])
        
    return np.mean(np.array(score),axis=0)


def evaluate_model_mean_all(X, y, model, stratify ,  test_size=0.33, loss = my_custom_loss_func, state = 100, size_features = 30):
    
    score = []
    for i in range(state):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=i, stratify=stratify)
        
        
        mask = (X_test['COUNTRY']==-1)
        X_test_FR = X_test[mask].copy()
        y_train = np.array(y_train)
        y_test_FR = np.array(y_test)[np.array(mask)].copy()
        
        
        mask = (X_test['COUNTRY']==1)
        X_test_DE = X_test[mask].copy()
        y_test_DE = np.array(y_test)[np.array(mask)].copy()
      
    
        X_train = np.array(X_train)
        X_test = np.array(X_test)
      
        X_test_FR = np.array(X_test_FR)
        X_test_DE = np.array(X_test_DE)
      
        #model_pca = PCA(n_components=X_train.shape[1])
        #model_pca.fit(X_train)
        #X_train = model_pca.transform(X_train)
        #X_test = model_pca.transform(X_test)
                
        
        model.fit(X_train,y_train)
        train_score = loss(y_train, model.predict(X_train))
        y_predict = model.predict(X_test)
        test_score = loss(y_test , y_predict.reshape(-1))
        
        y_predict_FR = model.predict(X_test_FR)
        test_score_FR = loss(np.array(y_test_FR) , y_predict_FR.reshape(-1))
        
        y_predict_DE = model.predict(X_test_DE)
        test_score_DE = loss(np.array(y_test_DE) , y_predict_DE.reshape(-1))
      
        score.append([test_score,test_score_FR,test_score_DE])
        
    return np.mean(np.array(score),axis=0)

In [31]:
path = 'data/'
X_train = pd.read_csv( path + 'X_train.csv')
Y_train = pd.read_csv(path + 'Y_train.csv')
X_test = pd.read_csv(path + 'X_test.csv')

In [32]:
data = pd.merge(X_train, Y_train, on=['ID'])
all_data = data.copy()
all_data['train'] = 1
all_data = pd.concat([all_data,X_test])
all_data['TARGET'] = all_data['TARGET'].fillna(0)
all_data['train'] = all_data['train'].fillna(0)

In [33]:
from sklearn.impute import KNNImputer

def regress_var(df, x_columns, y_column, model, out = True):
    temp = df.dropna().copy()
    X = np.array(temp[x_columns])
    y = np.array(temp[y_column])
    model.fit(X,y)
    if(out):
        print("Model score", model.score(X,y))
    return df.apply(lambda row: model.predict(np.array(row[x_columns]).reshape(-1,len(x_columns)))[0] if(np.isnan(row[y_column])) else row[y_column], axis=1)


def clean_knn(all_data_clean, k =5 ):
    imputer = KNNImputer(n_neighbors=k)
    remove_columns = ['ID','DAY_ID','TARGET','train']
    keep = all_data_clean.columns.difference(remove_columns)
    all_data_clean[keep] = imputer.fit_transform(all_data_clean[keep])
    return all_data_clean
    
def clean_regression(all_data_clean):
    
    #all_data_clean['DE_FR_EXCHANGE'] = all_data_clean['DE_FR_EXCHANGE'].fillna(0)
    #all_data_clean['FR_DE_EXCHANGE'] = all_data_clean['FR_DE_EXCHANGE'].fillna(0)
    
    all_data_clean['DE_FR_EXCHANGE'] = all_data_clean['DE_FR_EXCHANGE'].fillna(all_data_clean['DE_FR_EXCHANGE'].mean(numeric_only=True))
    all_data_clean['FR_DE_EXCHANGE'] = all_data_clean['FR_DE_EXCHANGE'].fillna(all_data_clean['FR_DE_EXCHANGE'].mean(numeric_only=True))
    
    x_columns = ['DE_FR_EXCHANGE']
    y_column = 'DE_NET_EXPORT'
    all_data_clean[y_column] = regress_var(all_data_clean, x_columns, y_column, LinearRegression(), out=True)
    all_data_clean['DE_NET_IMPORT'] = - all_data_clean['DE_NET_EXPORT']
    
    x_columns = ['FR_DE_EXCHANGE']
    y_column = 'FR_NET_EXPORT'
    all_data_clean[y_column] = regress_var(all_data_clean, x_columns, y_column, LinearRegression(), out=True)
    all_data_clean['FR_NET_IMPORT'] = - all_data_clean['FR_NET_EXPORT']
    
    all_data_clean = all_data_clean.fillna(all_data_clean.mean(numeric_only=True))
    
    return all_data_clean

def remove_features(all_data_clean):
    
    all_data_clean = all_data_clean.drop(['DE_FR_EXCHANGE', 'FR_NET_IMPORT','DE_NET_IMPORT'],axis=1)
    all_data_clean['FR_NET_EXPORT'] -= all_data_clean['FR_DE_EXCHANGE']
    all_data_clean['DE_NET_EXPORT'] += all_data_clean['FR_DE_EXCHANGE']
    
    
    all_data_clean['DE_CONSUMPTION_RENEWABLE'] = all_data_clean['DE_CONSUMPTION'] - all_data_clean['DE_RESIDUAL_LOAD']
    all_data_clean['FR_CONSUMPTION_RENEWABLE'] = all_data_clean['FR_CONSUMPTION'] - all_data_clean['FR_RESIDUAL_LOAD']

    
    #all_data_clean = all_data_clean.rename(columns={'DE_CONSUMPTION':'DE_CONSUMPTION_RENEWABLE', 'FR_CONSUMPTION':'FR_CONSUMPTION_RENEWABLE' })
    
    all_data_clean['DE_FLOW_GAS'] = all_data_clean['DE_GAS']*all_data_clean['GAS_RET']
    all_data_clean['DE_FLOW_COAL'] = all_data_clean['DE_COAL']*all_data_clean['COAL_RET']
    all_data_clean['DE_FLOW_LIGNITE'] = all_data_clean['DE_LIGNITE']*all_data_clean['CARBON_RET']


    all_data_clean['FR_FLOW_GAS'] = all_data_clean['FR_GAS']*all_data_clean['GAS_RET']
    all_data_clean['FR_FLOW_COAL'] = all_data_clean['FR_COAL']*all_data_clean['COAL_RET']
    
    return all_data_clean

def add_clusters(k, all_data_clean,cols,c):
    from sklearn.cluster import KMeans
    X_season = all_data_clean[cols]
    #all_data_clean = all_data_clean.drop(cols,axis=1)
    kmeans = KMeans(n_clusters=k,random_state = 0).fit(np.array(X_season))
    all_data_clean[c] = kmeans.predict(np.array(X_season))
    all_data_clean = pd.get_dummies(all_data_clean, columns=[c])
    return all_data_clean


def replace_outliers(all_data_clean, cols, threshold = 3):
    for c in cols :
        upper_limit = all_data_clean[c].mean() + threshold*all_data_clean[c].std()
        lower_limit = all_data_clean[c].mean() - threshold*all_data_clean[c].std()    
        all_data_clean[c] = np.where(
            (all_data_clean[c]>upper_limit) & (all_data_clean['train']==0),
            upper_limit,
            np.where(
                (all_data_clean[c]<lower_limit) & (all_data_clean['train']==0),
                lower_limit,
                all_data_clean[c]
            ))
    return all_data_clean

def remove_outliers(all_data_clean, cols, threshold = 3):
    for c in cols :
        upper_limit = all_data_clean[c].mean() + threshold*all_data_clean[c].std()
        lower_limit = all_data_clean[c].mean() - threshold*all_data_clean[c].std()    
        all_data_clean[c] = np.where(
            (all_data_clean[c]>upper_limit) & (all_data_clean['train']==1),
            np.nan,
            np.where(
                (all_data_clean[c]<lower_limit) & (all_data_clean['train']==1),
                np.nan,
                all_data_clean[c]
            ))
    all_data_clean = all_data_clean.dropna()
    return all_data_clean

In [34]:
all_data_clean = all_data.copy()
all_data_clean['COUNTRY'] = all_data_clean['COUNTRY'].apply(lambda x: 0 if x =='FR' else 1)


#all_data_clean = all_data_clean.apply(lambda x: x.fillna(x.median()),axis=0)

#clean
all_data_clean = clean_regression(all_data_clean)

#remove features
all_data_clean = remove_features(all_data_clean)


columns = ['FR_DE_EXCHANGE', 'DE_NET_EXPORT',
       'FR_NET_EXPORT', 'DE_GAS', 'FR_GAS', 'DE_COAL', 'FR_COAL', 'DE_HYDRO',
       'FR_HYDRO', 'DE_NUCLEAR', 'FR_NUCLEAR', 'DE_SOLAR', 'FR_SOLAR',
       'DE_WINDPOW', 'FR_WINDPOW', 'DE_LIGNITE', 'DE_RESIDUAL_LOAD',
       'FR_RESIDUAL_LOAD', 'GAS_RET', 'COAL_RET', 'CARBON_RET']

#all_data_clean = remove_outliers(all_data_clean, columns)

#all_data_clean = replace_outliers(all_data_clean, columns)

'''
cols = ['DE_RAIN','FR_RAIN','DE_WIND','FR_WIND','DE_TEMP','FR_TEMP']
all_data_clean = add_clusters(4,all_data_clean,cols,'season')



cols = [ 'FR_HYDRO', 'FR_NUCLEAR', 'FR_SOLAR', 'FR_WINDPOW']
all_data_clean = add_clusters(4,all_data_clean,cols,'E_FR')


cols = [ 'DE_HYDRO', 'DE_NUCLEAR', 'DE_SOLAR', 'DE_WINDPOW', 'DE_LIGNITE']
all_data_clean = add_clusters(4,all_data_clean,cols,'E_DE')


cols = ['FR_GAS', 'FR_COAL','FR_RESIDUAL_LOAD']
all_data_clean = add_clusters(4,all_data_clean,cols,'N_FR')

cols = ['DE_GAS', 'DE_COAL','DE_RESIDUAL_LOAD']
all_data_clean = add_clusters(4,all_data_clean,cols,'N_DE')

'''

Model score 0.49328444244850245
Model score 0.4424460230408356


"\ncols = ['DE_RAIN','FR_RAIN','DE_WIND','FR_WIND','DE_TEMP','FR_TEMP']\nall_data_clean = add_clusters(4,all_data_clean,cols,'season')\n\n\n\ncols = [ 'FR_HYDRO', 'FR_NUCLEAR', 'FR_SOLAR', 'FR_WINDPOW']\nall_data_clean = add_clusters(4,all_data_clean,cols,'E_FR')\n\n\ncols = [ 'DE_HYDRO', 'DE_NUCLEAR', 'DE_SOLAR', 'DE_WINDPOW', 'DE_LIGNITE']\nall_data_clean = add_clusters(4,all_data_clean,cols,'E_DE')\n\n\ncols = ['FR_GAS', 'FR_COAL','FR_RESIDUAL_LOAD']\nall_data_clean = add_clusters(4,all_data_clean,cols,'N_FR')\n\ncols = ['DE_GAS', 'DE_COAL','DE_RESIDUAL_LOAD']\nall_data_clean = add_clusters(4,all_data_clean,cols,'N_DE')\n\n"

In [35]:

all_data_clean["FR_PRODUCTION"] = all_data_clean['FR_GAS'] + all_data_clean['FR_COAL'] + all_data_clean['FR_HYDRO'] + all_data_clean['FR_NUCLEAR'] + all_data_clean['FR_SOLAR'] + all_data_clean['FR_WINDPOW']
all_data_clean["DE_PRODUCTION"] = all_data_clean['DE_GAS'] + all_data_clean['DE_COAL'] + all_data_clean['DE_HYDRO'] + all_data_clean['DE_NUCLEAR'] + all_data_clean['DE_SOLAR'] + all_data_clean['DE_WINDPOW'] + all_data_clean['DE_LIGNITE']

all_data_clean['FR_NEED'] = all_data_clean["FR_CONSUMPTION"] - all_data_clean['FR_PRODUCTION']  
all_data_clean['DE_NEED'] = all_data_clean["DE_CONSUMPTION"] - all_data_clean['DE_PRODUCTION'] 

all_data_clean['FR_NEED_RATIO'] =   all_data_clean['FR_CONSUMPTION'] / all_data_clean["FR_PRODUCTION"]
all_data_clean['DE_NEED_RATIO'] =  all_data_clean['DE_CONSUMPTION'] / all_data_clean["DE_PRODUCTION"] 

all_data_clean['FR_NEED_RATIO_1'] =   all_data_clean['FR_NEED'] / all_data_clean["FR_PRODUCTION"]
all_data_clean['DE_NEED_RATIO_1'] =  all_data_clean['DE_NEED'] / all_data_clean["DE_PRODUCTION"] 


all_data_clean['FR_NEED_RATIO_2'] =   all_data_clean["FR_PRODUCTION"] / all_data_clean['FR_NEED']
all_data_clean['DE_NEED_RATIO_2'] =   all_data_clean["DE_PRODUCTION"] / all_data_clean['DE_NEED'] 

all_data_clean['FR_NEED_RATIO_3'] =   all_data_clean["FR_PRODUCTION"] / all_data_clean['FR_CONSUMPTION']
all_data_clean['DE_NEED_RATIO_3'] =   all_data_clean["DE_PRODUCTION"] / all_data_clean['DE_CONSUMPTION'] 



In [36]:
X_train = all_data_clean[all_data_clean['train'] == 1].drop(['train','TARGET','ID','DAY_ID'],axis=1)
X_test = all_data_clean[all_data_clean['train'] == 0].drop(['train','TARGET','DAY_ID'],axis=1)
id_train = all_data_clean[all_data_clean['train'] == 1]['ID']
id_test = X_test['ID']
X_test = X_test.drop('ID',axis=1)
y_train = all_data_clean[all_data_clean['train'] == 1]['TARGET']


X_train_FR = all_data_clean[(all_data_clean['train'] == 1) & (all_data_clean['COUNTRY'] == 0)].drop(['train','TARGET','ID','DAY_ID','COUNTRY'],axis=1)
X_train_DE = all_data_clean[(all_data_clean['train'] == 1) & (all_data_clean['COUNTRY'] == 1)].drop(['train','TARGET','ID','DAY_ID','COUNTRY'],axis=1)
id_train_DE = all_data_clean[(all_data_clean['train'] == 1) & (all_data_clean['COUNTRY'] == 1)]['ID']
y_train_FR = all_data_clean[(all_data_clean['train'] == 1) & (all_data_clean['COUNTRY'] == 0)]['TARGET']
y_train_DE = all_data_clean[(all_data_clean['train'] == 1) & (all_data_clean['COUNTRY'] == 1)]['TARGET']

X_test_FR = all_data_clean[(all_data_clean['train'] == 0) & (all_data_clean['COUNTRY'] == 0)].drop(['train','TARGET','DAY_ID','COUNTRY'],axis=1)
id_test_FR = X_test_FR['ID']
X_test_FR = X_test_FR.drop('ID',axis=1)

X_test_DE = all_data_clean[(all_data_clean['train'] == 0) & (all_data_clean['COUNTRY'] == 1)].drop(['train','TARGET','DAY_ID','COUNTRY'],axis=1)
id_test_DE = X_test_DE['ID']
X_test_DE = X_test_DE.drop('ID',axis=1)

In [37]:
X_train['COUNTRY'] = 2*X_train['COUNTRY'] - 1 
X_test['COUNTRY'] = 2*X_test['COUNTRY'] - 1 


In [38]:
import tensorflow as tf
from fast_soft_sort.tf_ops import soft_rank, soft_sort
import torch

In [39]:

from scipy.optimize import minimize
import numpy as np

def mean_absolute_percentage_error(y_pred, y_true, sample_weights=None):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert len(y_true) == len(y_pred)
    
    if np.any(y_true==0):
        print("Found zeroes in y_true. MAPE undefined. Removing from set...")
        idx = np.where(y_true==0)
        y_true = np.delete(y_true, idx)
        y_pred = np.delete(y_pred, idx)
        if type(sample_weights) != type(None):
            sample_weights = np.array(sample_weights)
            sample_weights = np.delete(sample_weights, idx)
        
    if type(sample_weights) == type(None):
        return(np.mean(np.abs((y_true - y_pred) / y_true)) * 100)
    else:
        sample_weights = np.array(sample_weights)
        assert len(sample_weights) == len(y_true)
        return(100/sum(sample_weights)*np.dot(
                sample_weights, (np.abs((y_true - y_pred) / y_true))
        ))
 

from sklearn.base import BaseEstimator

class CustomLinearModel(BaseEstimator):
    """
    Linear model: Y = XB, fit by minimizing the provided loss_function
    with L2 regularization
    """
    def __init__(self, X=None, Y=None, sample_weights=None, beta_init=None, 
                 regularization=0.01, alpha = 1, optim='BFGS'):
        self.regularization = regularization
        self.beta = None
        self.sample_weights = sample_weights
        self.beta_init = beta_init
        self.alpha = alpha
        self.X = X
        self.Y = Y
        self.optim = optim
        self.coef_ = self.beta
    
       
    def clip_beta(self):
        self.beta[np.abs(self.beta) < 1e-4] = 0
    
    def cons_f(self,beta):
        return np.var(self.X@beta)
    def cons_J(x):
        return [[2*x[0], 1], [2*x[0], -1]]
    def cons_H(x, v):
        return v[0]*np.array([[2, 0], [0, 0]]) + v[1]*np.array([[2, 0], [0, 0]])
    
    def mean_absolute_error(self, y_pred, y_true, sample_weights=None):
        
        
        #values = tf.convert_to_tensor([y_pred], dtype=tf.float64)

        #y_pred = soft_rank(values, regularization_strength=1).numpy().reshape(-1)
        
        #values = tf.convert_to_tensor([y_true], dtype=tf.float64)
        
        #y_true = soft_rank(values, regularization_strength=1).numpy().reshape(-1)
        
        #l = nn.HuberLoss()
        #return l(torch.tensor([y_pred]),torch.tensor([y_true]))
        return np.mean(np.abs(y_pred-y_true)**(self.alpha))
    
    def score(self,X,y_true):
        y_pred = self.predict(X)
        return my_custom_loss_func(y_pred,y_true)
        
    
    def predict(self, X):
        prediction = X@self.beta
        return(prediction)

    def model_error(self):
        y_pred = self.predict(self.X)
        error = self.mean_absolute_error(
            y_pred, self.Y, sample_weights=self.sample_weights
        )
        return(error)
    
    def l2_regularized_loss(self, beta):
        self.beta = beta
        return(self.model_error() + \
               sum(self.regularization*np.abs(np.array(self.beta))**2))
    
    def l1_regularized_loss(self, beta):
        self.beta = beta
        return(self.model_error() + \
               sum(self.regularization*np.abs(np.array(self.beta))))
    
    
    def fit(self, X, Y, maxiter=10000):       
        self.X = np.array(X)
        self.Y = np.array(Y) 
        # Initialize beta estimates (you may need to normalize
        # your data and choose smarter initialization values
        # depending on the shape of your loss function)
        if type(self.beta_init)==type(None):
            # set beta_init = 1 for every feature
            self.beta_init = np.array([1]*self.X.shape[1])
        else: 
            self.beta_init = np.array([1]*self.X.shape[1])
            self.beta = None
            
        if self.beta!=None and all(self.beta_init == self.beta):
            print("Model already fit once; continuing fit with more itrations.")
        
        #bounds = [(-0.2,0.2)]*len(self.beta_init)
        res = minimize(self.l1_regularized_loss, self.beta_init, 
                       method=self.optim, options={'maxiter': maxiter,'disp':False}) #constraints=nonlinear_constraint)
        self.beta = res.x
        self.beta_init = self.beta
        self.clip_beta()
        self.coef_ = self.beta



In [40]:
def mean_square_error(y_pred,y_true, sample_weights=None):
    return np.mean((y_pred-y_true)**2)

def mean_absolute_error(y_pred,y_true, sample_weights=None):
    return np.mean(np.abs(y_pred-y_true))

def error_corr(y_pred,y_true, sample_weights = None):
    return  1 - spearmanr(y_true, y_pred).correlation
    #return np.mean((y_pred-y_true)**2) - np.mean(y_pred**2)

In [41]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2,include_bias = False)
X_train_augmented  = poly.fit_transform(X_train)
X_test_augmented = poly.transform(X_test)
X_train_augmented = pd.DataFrame(columns=poly.get_feature_names(X_train.columns),data = np.array(X_train_augmented))
X_test_augmented = pd.DataFrame(columns=poly.get_feature_names(X_train.columns),data = np.array(X_test_augmented))

In [42]:

cols = ['DE_NET_EXPORT',
 'DE_HYDRO',
 'FR_WINDPOW',
 'DE_RESIDUAL_LOAD',
  'DE_FLOW_COAL',
 'DE_FLOW_LIGNITE',
 'CARBON_RET',
 'DE_CONSUMPTION_RENEWABLE',
 'COUNTRY_DE_CONSUMPTION_RENEWABLE',
 'COUNTRY_CARBON_RET',
 'COUNTRY_DE_RESIDUAL_LOAD',
 ]

cols = ['DE_NET_EXPORT',
 'DE_RESIDUAL_LOAD',
 'DE_CONSUMPTION_RENEWABLE',
 'COUNTRY DE_RESIDUAL_LOAD',
 'DE_NET_EXPORT E_DE_0',
 'DE_NET_EXPORT E_DE_3',
 'DE_NET_EXPORT N_DE_1',
 'FR_NET_EXPORT DE_WIND',
 'FR_COAL FR_CONSUMPTION_RENEWABLE',
 'FR_HYDRO N_FR_2',
 'DE_SOLAR DE_FLOW_LIGNITE',
 'FR_SOLAR E_FR_0',
 'FR_SOLAR E_FR_1',
 'DE_LIGNITE N_DE_0',
 'GAS_RET CARBON_RET',
 'DE_FLOW_LIGNITE FR_FLOW_GAS',
 'FR_FLOW_GAS FR_NEED_RATIO',
 'FR_FLOW_COAL N_DE_1',
 'E_FR_0 FR_NEED_RATIO',
 'N_FR_0 FR_NEED_RATIO']

#cols =  keep




cols = ['DE_NET_EXPORT',
 'COUNTRY DE_RESIDUAL_LOAD',
 'DE_NET_EXPORT E_DE_1',
 'FR_NET_EXPORT DE_WIND',
 'FR_GAS E_DE_0',
 'DE_LIGNITE N_DE_3',
 'DE_WIND GAS_RET',
 'FR_WIND DE_FLOW_LIGNITE',
 'DE_CONSUMPTION_RENEWABLE N_FR_3',
 'DE_CONSUMPTION_RENEWABLE N_DE_0',
 'FR_FLOW_GAS FR_NEED_RATIO_1',
 'FR_FLOW_COAL N_DE_0',
 'FR_FLOW_COAL FR_NEED_RATIO',]


cols = ['DE_NET_EXPORT',
 'DE_HYDRO',
 'FR_WINDPOW',
 'DE_RESIDUAL_LOAD',
  'DE_FLOW_COAL',
 'DE_FLOW_LIGNITE',
 'CARBON_RET',
 'DE_CONSUMPTION_RENEWABLE',
 'COUNTRY DE_CONSUMPTION_RENEWABLE',
 'COUNTRY CARBON_RET',
 'COUNTRY DE_RESIDUAL_LOAD',
 ]




X_train_COUNTRY = X_train_augmented['COUNTRY']
#X_train_augmented = X_train_augmented[cols]
#X_test_augmented = X_test_augmented[cols]


In [43]:
X = pd.concat([X_train_augmented,X_test_augmented])
#X = X.drop(['COUNTRY'],axis=1)

scaler = StandardScaler()
scaler.fit(X)
cols = X.columns
X_train_augmented[cols] = scaler.transform(X_train_augmented[cols])
X_test_augmented[cols] = scaler.transform(X_test_augmented[cols])


#X = pd.concat([X_train_augmented,X_test_augmented])
#model = PCA(n_components=X.shape[1])
#model.fit(X)
#X_train_augmented = model.transform(X_train_augmented)
#X_test_augmented = model.transform(X_test_augmented)

In [44]:
X_test_augmented.columns

Index(['COUNTRY', 'DE_CONSUMPTION', 'FR_CONSUMPTION', 'FR_DE_EXCHANGE',
       'DE_NET_EXPORT', 'FR_NET_EXPORT', 'DE_GAS', 'FR_GAS', 'DE_COAL',
       'FR_COAL',
       ...
       'FR_NEED_RATIO_2^2', 'FR_NEED_RATIO_2 DE_NEED_RATIO_2',
       'FR_NEED_RATIO_2 FR_NEED_RATIO_3', 'FR_NEED_RATIO_2 DE_NEED_RATIO_3',
       'DE_NEED_RATIO_2^2', 'DE_NEED_RATIO_2 FR_NEED_RATIO_3',
       'DE_NEED_RATIO_2 DE_NEED_RATIO_3', 'FR_NEED_RATIO_3^2',
       'FR_NEED_RATIO_3 DE_NEED_RATIO_3', 'DE_NEED_RATIO_3^2'],
      dtype='object', length=1274)

In [45]:
from lazypredict.Supervised import LazyRegressor
offset = int(X_train_augmented.shape[0] * 0.8)

X_train_m, y_train_m = X_train_augmented[:offset], y_train[:offset]
X_test_m, y_test_m = X_train_augmented[offset:], y_train[offset:]

reg = LazyRegressor(verbose=1, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train_m, X_test_m, y_train_m, y_test_m)
print(models)

  2%|▏         | 1/42 [00:09<06:45,  9.90s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': -0.24852749941652896, 'Adjusted R-Squared': 1.3812102406005384, 'RMSE': 1.0296925246251663, 'Time taken': 9.896729707717896}


  5%|▍         | 2/42 [00:26<09:08, 13.71s/it]

{'Model': 'BaggingRegressor', 'R-Squared': -0.46613604202419645, 'Adjusted R-Squared': 1.4476521931590272, 'RMSE': 1.1158238469851123, 'Time taken': 16.369807720184326}


  7%|▋         | 3/42 [00:28<05:26,  8.38s/it]

...

{'Model': 'TweedieRegressor', 'R-Squared': -0.12424797856444814, 'Adjusted R-Squared': 1.343264239356768, 'RMSE': 0.9771012331153692, 'Time taken': 0.29684948921203613}


 98%|█████████▊| 41/42 [1:43:09<01:36, 96.21s/it] 

{'Model': 'XGBRegressor', 'R-Squared': -0.4294813886636073, 'Adjusted R-Squared': 1.436460505964913, 'RMSE': 1.1017872855430424, 'Time taken': 6.778566360473633}


100%|██████████| 42/42 [1:43:14<00:00, 147.49s/it]

{'Model': 'LGBMRegressor', 'R-Squared': -0.34074029811397244, 'Adjusted R-Squared': 1.4093653779077497, 'RMSE': 1.067040331965073, 'Time taken': 5.305518627166748}
                                                              Adjusted R-Squared  \
Model                                                                              
QuantileRegressor             69749104118478153119995410593493848251829302250...   
SGDRegressor                                        4903885512387801118670848.00   
LinearRegression                                         82630111694875213824.00   
TransformedTargetRegressor                               82630111694875213824.00   
Lars                                                                     2050.29   
Ridge                                                                       2.42   
KernelRidge                                                                 2.42   
RidgeCV                                                                     2.07